In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd

#diretorio = r'Bases\Base_atualizada/'
diretorio = r'C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\Manipulacao_arquivos\Manipulacao_Arquivos_Python\Bases/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
ESTABELE = ['CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'MATRIZ_FILIAL', 'NOME_FANTASIA',
       'SITUACAO_CADASTRAL', 'DATA_SITUACAO_CADASTRAL',
       'MOTIVO_SITUACAO_CADASTRAL', 'DATA_INICIO_ATIVIDADE', 'CNAE_PRINCIPAL',
       'CNAE_SECUNDARIO', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO',
       'COMPLEMENTO', 'BAIRRO', 'CEP', 'UF', 'MUNICIPIO', 'TELEFONE1',
       'TELEFONE2', 'FAX', 'EMAIL']

dtypes = {'CNPJ_BASE': 'category',
 'CNPJ_ORDEM': 'category',
 'CNPJ_DV': 'category'}

CNAES = {5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        5612100: 'Serviços ambulantes de alimentação'}

lista_cnae = []
lista_descricao = []
for cnae in CNAES.keys():
    lista_cnae.append(cnae)
for descricao in CNAES.values():
    lista_descricao.append(descricao)
lista_descricao

['Restaurantes e similares',
 'Lanchonetes casas de chá de sucos e similares',
 'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
 'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
 'Serviços ambulantes de alimentação']

In [3]:
c = 0
for file in all_files:
    dados =pd.read_csv(f'{diretorio}{file}', names=ESTABELE, dtype=dtypes, sep=';')
    dados['CNAE_DESCRICAO'] = lista_descricao[c]
    dados = dados[[
        'CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'NOME_FANTASIA', 'CNAE_PRINCIPAL','CNAE_DESCRICAO',
        'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO',
       'COMPLEMENTO', 'BAIRRO', 'CEP', 'UF', 'MUNICIPIO', 'TELEFONE1']]
    c +=1
    dados.to_csv(f"base_csv_estabelecimentos/{file}", mode='w',index=False, sep=';')
dados.head(2)

,CNPJ_BASE,CNPJ_ORDEM,CNPJ_DV,NOME_FANTASIA,CNAE_PRINCIPAL,CNAE_DESCRICAO,TIPO_LOGRADOURO,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CEP,UF,MUNICIPIO,TELEFONE1
0,35465067,0001,86,--empty--,5612100,Serviços ambulantes de alimentação,PRACA,JEOVA CAETANO,175,CASA,JARDIM ESPERANCA,57307-590,AL,2705,+558281821926
1,35472474,0001,10,--empty--,5612100,Serviços ambulantes de alimentação,RUA,ALCEBIADES CUNHA,16,CASA,CENTRO,45350-000,BA,3639,+557335381200


In [4]:
dados.columns

Index(['CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'NOME_FANTASIA', 'CNAE_PRINCIPAL',
       'CNAE_DESCRICAO', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO',
       'COMPLEMENTO', 'BAIRRO', 'CEP', 'UF', 'MUNICIPIO', 'TELEFONE1'],
      dtype='object')

## Com Pandas

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
import glob

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

path = f'{diretorio_estabelecimentos}/*.csv'
all_files_estabelecimentos = glob.glob(path)

lista_df_estabelecimentos = []
for file in all_files_estabelecimentos:
    d_estabelecimento = pd.read_csv(file, 
    sep=';', 
    dtype=dtypes_ESTABELE, 
    names=ESTABELE)
    lista_df_estabelecimentos.append(d_estabelecimento)

estabelecimentos = pd.concat(lista_df_estabelecimentos, axis=0, ignore_index=True)

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

path = f'{diretorio_empresa}/*.csv'
all_files_empresa = glob.glob(path)

lista_df_empresas = []
for file in all_files_empresa:
    d_empresa = pd.read_csv(file,
        sep=';',
        dtype=dtypes_EMPRESA, 
        names=EMPRESA)
    #lista_df_empresas.append(d_empresa)

    #empresas = pd.concat(lista_df_empresas, axis=0, ignore_index=True)

    merged_data = pd.merge(estabelecimentos, d_empresa, on='CNPJ_BASE')

    chunk_size = 1000000
    for i in range(0, len(merged_data), chunk_size):
        df_chunk = merged_data.iloc[i:i+chunk_size]
        df_chunk.to_csv(f'merge_base/merged_data_{i}.csv', mode='a', index=False)

## Com Dask

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import dask.dataframe as dd
import pandas as pd

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

# Lendo os arquivos
estabelecimentos = dd.read_csv(diretorio_estabelecimentos + '/*.csv', delimiter=';',  dtype=dtypes_ESTABELE, names=ESTABELE)
empresas = dd.read_csv(diretorio_empresa + '/*.csv', delimiter=';', dtype=dtypes_EMPRESA, names=EMPRESA)

# unindo as tabelas
merged = estabelecimentos.merge(empresas, on='CNPJ_BASE')

merged.to_csv('merge_base/merged_data.csv', single_file=True)

# mostrando as primeiras linhas
print(merged.head())

## Com Vaex

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import vaex

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

# Lendo os arquivos
estabelecimentos = vaex.open(diretorio_estabelecimentos + '/*.csv', delimiter=';')
empresas = vaex.open(diretorio_empresa + '/*.csv', delimiter=';')

# unindo as tabelas
merged = estabelecimentos.join(empresas, on='CNPJ_BASE')

# mostrando as primeiras linhas
print(merged.head())

In [7]:
import pandas as pd
import json
dados = pd.read_csv(r'merge_base\merged_data_0.csv')
municipios = pd.read_csv('Municipios\municipios.csv', sep=';', names=['MUNICIPIO','CIDADE'])
dados = dados[['CNPJ_BASE', 'CNPJ', 'NOME_FANTASIA', 'RAZAO_SOCIAL', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO', 'COMPLEMENTO', 'BAIRRO',
       'CEP', 'UF', 'MUNICIPIO']]
merged_data = pd.merge(dados, municipios, on='MUNICIPIO')
merged_data.drop(columns='MUNICIPIO', inplace=True)
merged_data.to_csv(r'merge_base\merged_data_0.csv', index=False)


In [4]:
#diretorio = r'Bases\Base_atualizada/'
import time
import pandas as pd
import json
base = pd.DataFrame()
diretorio = r'merge_base/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))
for file in all_files:
    dados = pd.read_csv(f'{diretorio}{file}', sep=';' )
    dados = dados.sort_values(by=['CNPJ','TELEFONE'], ascending=[True,False])
    dados['TELEFONE'] = dados['TELEFONE'].astype(str)
    dados['TELEFONE'] = dados['TELEFONE'].str.replace('.0', '', regex=True)
    dados['TELEFONE'] = dados['TELEFONE'].str.replace('(', '', regex=True)
    dados['TELEFONE'] = dados['TELEFONE'].str.replace(')', '', regex=True)
    base = pd.concat([base, dados[0:2000]], ignore_index=True)
base.to_csv("DB_SHOW_APPLE.csv",mode='w', index=False)
dados.head(2)
    

C:\Users\ABRASEL NACIONAL\AppData\Local\Temp\ipykernel_14936\585783564.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv(f'{diretorio}{file}', sep=';' )


,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,TELEFONE,EMAIL,CNAE_PRINCIPAL,CNAE_DESCRICAO
62464,1811000107,TITAN REPRESENTACOES LTDA,Indisponível,RUA SAO SEBASTIAO,219,NaN,CENTRO,JARDINOPOLIS,SP,14680000.0,nannan,NaN,5612100,Serviços ambulantes de alimentação
62617,5838000178,JOSE ANTONIO FERNANDES - DOCES,Indisponível,TRAVESSA SAO MATHEUS,113,NaN,SUMAREZINHO,RIBEIRAO PRETO,SP,14055564.0,1639612889,creusa.maria.barbosa@terra.com.br,5612100,Serviços ambulantes de alimentação
